In [3]:
import requests
import time
from datetime import datetime, timedelta
import json

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

25/02/23 20:59:45 WARN Utils: Your hostname, MacBook-Air-Pavel.local resolves to a loopback address: 127.0.0.1; using 192.168.1.66 instead (on interface en0)
25/02/23 20:59:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 20:59:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/02/24 02:05:31 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3336066 ms exceeds timeout 120000 ms
25/02/24 02:05:32 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/24 02:05:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

In [26]:
pyspark.__file__

'/opt/anaconda3/lib/python3.11/site-packages/pyspark/__init__.py'

In [21]:
spark = SparkSession.builder.master("local[*]") \
    .appName("spark_uploader") \
    .config("spark.jars", "/Users/nomad/Downloads/clickhouse-jdbc-0.8.0.jar") \
    .getOrCreate()

25/02/16 22:25:09 WARN Utils: Your hostname, MacBook-Air-Pavel.local resolves to a loopback address: 127.0.0.1; using 192.168.1.65 instead (on interface en0)
25/02/16 22:25:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/02/16 22:25:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Exception in thread "main" java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.hadoop.fs.FileSystem$Cache$Key.<init>(FileSystem.java:3746)
	at org.apache.hadoop.fs.FileSystem$Cache$Key.<init>(FileSystem.java:3736)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3520)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.spark.util.Depende

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
clickhouse_url = "jdbc:clickhouse://192.168.1.66:9000/metrics"
clickhouse_properties = {
    "user": "root",
    "password": "020491",
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

In [14]:
!spark-shell --version

25/02/16 21:46:00 WARN Utils: Your hostname, MacBook-Air-Pavel.local resolves to a loopback address: 127.0.0.1; using 192.168.1.65 instead (on interface en0)
25/02/16 21:46:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.4
      /_/
                        
Using Scala version 2.12.18, Java HotSpot(TM) 64-Bit Server VM, 23.0.2
Branch HEAD
Compiled by user yangjie01 on 2024-12-17T04:51:46Z
Revision a6f220d951742f4074b37772485ee0ec7a774e7d
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
oauth_token = 'y0_AgAAAAAisvRKAAutVAAAAAEC7JiLAAAtHPjoix1FhahM1KuvZy5qiQ3ZvQ'

In [5]:
# Настройки
url = 'https://api.appmetrica.yandex.ru/logs/v1/export/events.json'
headers = {'Authorization': f'OAuth {oauth_token}'}
application_id = '3757420'
fields = 'event_datetime,event_name,event_json,appmetrica_device_id,session_id'  # Укажите нужные поля

In [7]:
date_since = datetime(2023, 12, 5)  # Начальная дата
date_until = datetime(2023, 12, 10)  # Конечная дата

start_date = date_since
while start_date <= date_until:
    fetch_data_for_day(start_date)
    start_date += timedelta(days=1)

: 

In [6]:
# Функция для получения данных за один день
def fetch_data_for_day(date):
    params = {
        'application_id': application_id,
        'date_since': date.strftime('%Y-%m-%d'),
        'date_until': date.strftime('%Y-%m-%d'),
        'fields': fields
    }
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 202:
        print(f"Ожидаю 200 для даты {date}...")
        while response.status_code == 202:
            print(response.status_code)
            time.sleep(15)
            response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        print(f"Загружаю данные за {date}...")
        response = json.loads(response.text)
        parts = response['data']
        df = pd.DataFrame.from_dict(parts)
        df['event_datetime'] = pd.to_datetime(df['event_datetime'])
        df['event_json'] = df['event_json'].fillna('')
        df['event_name'] = df['event_name'].fillna('')
        df['appmetrica_device_id'] = df['appmetrica_device_id'].fillna('')
        df['session_id'] = df['session_id'].fillna('')
        
        batch_size = 100  # Размер пакета
        for i in range(0, len(df), batch_size):
            batch = df[i:i + batch_size]
            batch.to_sql(
                name='metrica_events',
                con=engine,
                if_exists='append',
                index=False
                )
        del response
        del parts
        del df
        gc.collect()

    else:
        print(f"Ошибка для даты {date}: {response.status_code}")

In [ ]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("ClickHouse Example") \
    .config("spark.jars", "/path/to/clickhouse-jdbc.jar") \
    .getOrCreate()

# Укажите параметры подключения к ClickHouse
clickhouse_url = "jdbc:clickhouse://<clickhouse_host>:<clickhouse_port>/<database>"
clickhouse_properties = {
    "user": "<username>",
    "password": "<password>",
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

# Пример DataFrame, который мы хотим записать в ClickHouse
data = [("Alice", 34), ("Bob", 45)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)

# Записываем DataFrame в таблицу ClickHouse
df.write \
    .format("jdbc") \
    .option("url", clickhouse_url) \
    .option("dbtable", "<table_name>") \
    .option("user", clickhouse_properties["user"]) \
    .option("password", clickhouse_properties["password"]) \
    .option("driver", clickhouse_properties["driver"]) \
    .mode("append") \
    .save()

# Останавливаем SparkSession
spark.stop()